### Exploring the Data

1. Load your data in using Pandas and start to explore. Save all of your early exploration code here and include in your final submission.

Problems noticed during exploratory analysis:
 * Some weeks are listed as floats instead of integers. That **will** need to be changed. Or the integers need to be changed to floats?
 * <s>Artist name is inverted. There should be a pretty simple formula to un-invert it so that it reads properly</s>
 * <s>'time' should be called 'length'</s>
 * It would be great to add a few columns that contain arithmetic functions
     * Something that calculates how long it took a track to peak
     * Something that calculates how long between something's peak and it's disappearance from charts
     * Something that calculates the lifespan of a track. 
 * There are no values in columns after the 67th week, perhaps earlier. Figuring out where the data stops might be tricky. I can just go through and delete the columns I know are empty, and that should allow me to run a describe again with the new final data. _Created a for loop to solve this problem instead_
 * Track length is an object, it shoudl be a number that math can be performed on. The same goes for date entered and date-peaked

In [37]:
import pandas as pd
import numpy as np

billboard = pd.read_csv('billboard.csv')
# I've commented them out for ease of running, but the exploratory work I did is below.
# print billboard.head()
# print billboard.tail()
print billboard.describe()
# This for loop iterates through the column names and removes any column that is empty.
# This can be done because no track was on the billboard list for longer than 65 weeks.
for x in billboard.columns:
    if billboard[x].count() == 0:
        billboard.drop(x,1,inplace=True)

# This goes through and changes the column heading for the week by week ranking so that
# there is no leading x.
for x in billboard.columns:
    if x.startswith('x'):
        billboard.rename(columns=lambda x: x.replace('x', ''), inplace=True)
        #print x[1:]

# I'm going to work here on getting artist names in their proper form


for x in billboard['artist.inverted']:
    if ',' in x:
        comma_loc = x.index(',')
        proper_name = x[comma_loc+2:]+' '+x[:comma_loc]
        billboard['artist.inverted'].replace(x, proper_name, inplace = True)

# If the below command weren't commented out
#print billboard['artist.inverted']

# This line changes a few of the column titles to be more appropriately named
billboard.rename(columns={'artist.inverted': 'artist', 'time': 'length_in_secs'}, inplace=True)
print billboard

# Change the first week type to be float in order to make it more compatible with the
# rest of the data
billboard['1st.week'] = billboard['1st.week'].astype(float)

print billboard.dtypes

         year   x1st.week   x2nd.week   x3rd.week   x4th.week   x5th.week  \
count   317.0  317.000000  312.000000  307.000000  300.000000  292.000000   
mean   2000.0   79.958991   71.173077   65.045603   59.763333   56.339041   
std       0.0   14.686865   18.200443   20.752302   22.324619   23.780022   
min    2000.0   15.000000    8.000000    6.000000    5.000000    2.000000   
25%    2000.0   74.000000   63.000000   53.000000   44.750000   38.750000   
50%    2000.0   81.000000   73.000000   66.000000   61.000000   57.000000   
75%    2000.0   91.000000   84.000000   79.000000   76.000000   73.250000   
max    2000.0  100.000000  100.000000  100.000000  100.000000  100.000000   

        x6th.week   x7th.week   x8th.week   x9th.week     ...      x67th.week  \
count  280.000000  269.000000  260.000000  253.000000     ...             0.0   
mean    52.360714   49.219331   47.119231   46.343874     ...             NaN   
std     24.473273   25.654279   26.370782   27.136419     ...  

In [38]:
# Convert the track length to seconds

lengths =[]
for x in billboard['length_in_secs']:
    col_spot = x.index(':')
    track_length = int(x[:col_spot])*60+int(x[col_spot+1:])    
    lengths.append(track_length)

billboard['length_in_secs'] = lengths


In [39]:
print billboard.describe()

         year  length_in_secs    1st.week    2nd.week    3rd.week    4th.week  \
count   317.0      317.000000  317.000000  312.000000  307.000000  300.000000   
mean   2000.0      242.425868   79.958991   71.173077   65.045603   59.763333   
std       0.0       42.401618   14.686865   18.200443   20.752302   22.324619   
min    2000.0      156.000000   15.000000    8.000000    6.000000    5.000000   
25%    2000.0      219.000000   74.000000   63.000000   53.000000   44.750000   
50%    2000.0      236.000000   81.000000   73.000000   66.000000   61.000000   
75%    2000.0      257.000000   91.000000   84.000000   79.000000   76.000000   
max    2000.0      470.000000  100.000000  100.000000  100.000000  100.000000   

         5th.week    6th.week    7th.week    8th.week    ...      56th.week  \
count  292.000000  280.000000  269.000000  260.000000    ...       2.000000   
mean    56.339041   52.360714   49.219331   47.119231    ...      25.500000   
std     23.780022   24.473273   2

In [40]:
# This codeblock translates the data in the date fields to be date times

for x in billboard['date.peaked']:
    date_x = pd.to_datetime(x)
    billboard['date.peaked'].replace(x, date_x, inplace=True)

for x in billboard['date.entered']:
    date_x = pd.to_datetime(x)
    billboard['date.entered'].replace(x, date_x, inplace=True)
# Here I created a new field called 'time_to_peak' that calculates how long it took a track
# to 'peak' on the billboard top 100
billboard['time_to_peak'] = billboard['date.peaked'] - billboard['date.entered']

#print billboard['time_to_peak']

In [41]:
# What types of data are we dealing with

print billboard.dtypes

year                        int64
artist                     object
track                      object
length_in_secs              int64
genre                      object
date.entered       datetime64[ns]
date.peaked        datetime64[ns]
1st.week                  float64
2nd.week                  float64
3rd.week                  float64
4th.week                  float64
5th.week                  float64
6th.week                  float64
7th.week                  float64
8th.week                  float64
9th.week                  float64
10th.week                 float64
11th.week                 float64
12th.week                 float64
13th.week                 float64
14th.week                 float64
15th.week                 float64
16th.week                 float64
17th.week                 float64
18th.week                 float64
19th.week                 float64
20th.week                 float64
21st.week                 float64
22nd.week                 float64
23rd.week     

In [42]:
#Let's look at some of the artists who popped up frequently on the top 100 that year
billboard['artist'].unique
print pd.value_counts(billboard['artist'].values, sort=True).head(10)

Jay-Z                  5
The Dixie Chicks       4
Whitney Houston        4
The Backstreet Boys    3
DMX                    3
Alan Jackson           3
SheDaisy               3
Sisqo                  3
Ricky Martin           3
Kelly Price            3
dtype: int64


In [43]:
# This sums across the rows, using the code in the block below, we should be able to do this for
# a specific range of columns
billboard.sum(1)

0      2633.0
1      2531.0
2      2820.0
3      2548.0
4      2637.0
5      2723.0
6      2939.0
7      2533.0
8      2864.0
9      3735.0
10     2732.0
11     2661.0
12     3120.0
13     3822.0
14     2776.0
15     2979.0
16     2994.0
17     3464.0
18     2819.0
19     2976.0
20     3038.0
21     2981.0
22     2957.0
23     2962.0
24     3636.0
25     2782.0
26     2771.0
27     2756.0
28     3230.0
29     3184.0
        ...  
287    2465.0
288    4077.0
289    2679.0
290    3917.0
291    2532.0
292    3084.0
293    2804.0
294    2719.0
295    3393.0
296    2778.0
297    2909.0
298    2607.0
299    2691.0
300    3056.0
301    2525.0
302    2564.0
303    2944.0
304    2709.0
305    2745.0
306    2594.0
307    2536.0
308    2558.0
309    2442.0
310    2437.0
311    2311.0
312    2282.0
313    2634.0
314    2408.0
315    2436.0
316    2301.0
dtype: float64

In [44]:
# This shows all the stuff that Jay-Z has done, and the average place his tracks were on a given week
print billboard.loc[billboard['artist'] == 'Jay-Z'].count()
#  Using this output, we can start to get a sense of when most artists tend to drop off the
# top 100
billboard.ix[:,7:].count()

year              5
artist            5
track             5
length_in_secs    5
genre             5
date.entered      5
date.peaked       5
1st.week          5
2nd.week          5
3rd.week          5
4th.week          5
5th.week          5
6th.week          5
7th.week          5
8th.week          5
9th.week          4
10th.week         3
11th.week         3
12th.week         3
13th.week         3
14th.week         3
15th.week         3
16th.week         2
17th.week         2
18th.week         2
19th.week         2
20th.week         2
21st.week         1
22nd.week         0
23rd.week         0
                 ..
37th.week         0
38th.week         0
39th.week         0
40th.week         0
41st.week         0
42nd.week         0
43rd.week         0
44th.week         0
45th.week         0
46th.week         0
47th.week         0
48th.week         0
49th.week         0
50th.week         0
51st.week         0
52nd.week         0
53rd.week         0
54th.week         0
55th.week         0


1st.week        317
2nd.week        312
3rd.week        307
4th.week        300
5th.week        292
6th.week        280
7th.week        269
8th.week        260
9th.week        253
10th.week       244
11th.week       236
12th.week       222
13th.week       210
14th.week       204
15th.week       197
16th.week       182
17th.week       177
18th.week       166
19th.week       156
20th.week       146
21st.week        65
22nd.week        55
23rd.week        48
24th.week        46
25th.week        38
26th.week        36
27th.week        29
28th.week        24
29th.week        20
30th.week        20
               ... 
37th.week         9
38th.week         8
39th.week         8
40th.week         7
41st.week         7
42nd.week         6
43rd.week         6
44th.week         6
45th.week         5
46th.week         5
47th.week         5
48th.week         4
49th.week         4
50th.week         4
51st.week         4
52nd.week         4
53rd.week         4
54th.week         2
55th.week         2


In [45]:
#billboard[weeks_on].apply(lambda s: (s > 0).count(), axis=1)

In [102]:
long = pd.melt(billboard, id_vars = ['artist', 'track'], value_vars = ['1st.week', '2nd.week', '3rd.week', '4th.week'])
print long
long.set_index('track', inplace=True)
print long



                  artist                                              track  \
0        Destiny's Child                           Independent Women Part I   
1                Santana                                       Maria, Maria   
2          Savage Garden                                 I Knew I Loved You   
3                Madonna                                              Music   
4     Christina Aguilera              Come On Over Baby (All I Want Is You)   
5                  Janet                              Doesn't Really Matter   
6        Destiny's Child                                        Say My Name   
7       Enrique Iglesias                                        Be With You   
8                  Sisqo                                         Incomplete   
9               Lonestar                                             Amazed   
10                N'Sync                                   It's Gonna Be Me   
11    Christina Aguilera                            

In [103]:
# Change the billboard index to be governed by the track names
#billboard = billboard.set_index('track')


KeyError: 'Got Beef'

In [48]:
billboard.describe()

,year,length_in_secs,1st.week,2nd.week,3rd.week,4th.week,5th.week,6th.week,7th.week,8th.week,...,57th.week,58th.week,59th.week,60th.week,61st.week,62nd.week,63rd.week,64th.week,65th.week,time_to_peak
count,317.0,317.000000,317.000000,312.000000,307.000000,300.000000,292.000000,280.000000,269.000000,260.000000,...,2.00000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.0,1.0,317
mean,2000.0,242.425868,79.958991,71.173077,65.045603,59.763333,56.339041,52.360714,49.219331,47.119231,...,27.50000,31.500000,35.500000,38.000000,42.500000,44.500000,47.500000,50.0,49.0,52 days 05:54:19.305993
std,0.0,42.401618,14.686865,18.200443,20.752302,22.324619,23.780022,24.473273,25.654279,26.370782,...,2.12132,0.707107,4.949747,1.414214,0.707107,3.535534,3.535534,0.0,NaN,40 days 20:49:20.696756
min,2000.0,156.000000,15.000000,8.000000,6.000000,5.000000,2.000000,1.000000,1.000000,1.000000,...,26.00000,31.000000,32.000000,37.000000,42.000000,42.000000,45.000000,50.0,49.0,0 days 00:00:00
25%,2000.0,219.000000,74.000000,63.000000,53.000000,44.750000,38.750000,33.750000,30.000000,27.000000,...,26.75000,31.250000,33.750000,37.500000,42.250000,43.250000,46.250000,50.0,49.0,21 days 00:00:00
50%,2000.0,236.000000,81.000000,73.000000,66.000000,61.000000,57.000000,51.500000,47.000000,45.500000,...,27.50000,31.500000,35.500000,38.000000,42.500000,44.500000,47.500000,50.0,49.0,49 days 00:00:00
75%,2000.0,257.000000,91.000000,84.000000,79.000000,76.000000,73.250000,72.250000,67.000000,67.000000,...,28.25000,31.750000,37.250000,38.500000,42.750000,45.750000,48.750000,50.0,49.0,70 days 00:00:00
max,2000.0,470.000000,100.000000,100.000000,100.000000,100.000000,100.000000,99.000000,100.000000,99.000000,...,29.00000,32.000000,39.000000,39.000000,43.000000,47.000000,50.000000,50.0,49.0,315 days 00:00:00


In [49]:
print billboard

     year              artist  \
0    2000     Destiny's Child   
1    2000             Santana   
2    2000       Savage Garden   
3    2000             Madonna   
4    2000  Christina Aguilera   
5    2000               Janet   
6    2000     Destiny's Child   
7    2000    Enrique Iglesias   
8    2000               Sisqo   
9    2000            Lonestar   
10   2000              N'Sync   
11   2000  Christina Aguilera   
12   2000    Vertical Horizon   
13   2000               Creed   
14   2000             Aaliyah   
15   2000     matchbox twenty   
16   2000        Mariah Carey   
17   2000          Faith Hill   
18   2000                 Mya   
19   2000               Dream   
20   2000        Toni Braxton   
21   2000        Marc Anthony   
22   2000                 98�   
23   2000     Destiny's Child   
24   2000        3 Doors Down   
25   2000               Sisqo   
26   2000  Christina Aguilera   
27   2000     Whitney Houston   
28   2000                 Joe   
29   2000 